<a href="https://colab.research.google.com/github/Kaisano/GCollab_torrentDL/blob/main/torrent_dl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Dependencies

In [1]:
!apt install 7zip
!python -m pip install --upgrade pip setuptools wheel
!pip install libtorrent lbry-libtorrent dropbox tomlkit
!curl https://rclone.org/install.sh | sudo bash
# !python -m pip install lbry-libtorrent

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  7zip
0 upgraded, 1 newly installed, 0 to remove and 41 not upgraded.
Need to get 971 kB of archives.
After this operation, 2,454 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 7zip amd64 21.07+dfsg-4 [971 kB]
Fetched 971 kB in 0s (7,558 kB/s)
Selecting previously unselected package 7zip.
(Reading database ... 121713 files and directories currently installed.)
Preparing to unpack .../7zip_21.07+dfsg-4_amd64.deb ...
Unpacking 7zip (21.07+dfsg-4) ...
Setting up 7zip (21.07+dfsg-4) ...
Processing triggers for man-db (2.10.2-1) ...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 58.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 57.0 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.2.0
    Uninstalling setuptools

# Imports
rich.progress for concurrent progress tracking<br>
libtorrent as torrent client<br>
dropbox as storage solution<br>

In [2]:
import sys
import os, subprocess, glob
from concurrent.futures import ThreadPoolExecutor, Future, as_completed

import logging
import time
from timeit import default_timer as timer

from rich.progress import Progress, TextColumn, BarColumn, TaskProgressColumn, TimeRemainingColumn
from pathlib import Path

from tomlkit import document, table, nl, comment
from tomlkit import dumps
from tomlkit.toml_file import TOMLFile

import libtorrent as lt

# Setup Filesystem

In [3]:
magnet_path = Path("/content/magnet.links")
magnet_file = TOMLFile(magnet_path.as_posix())

if not magnet_path.exists():
  magnet_document = document()

  example_folder_tor = table()
  example_folder_tor["magnets"] = ["mag1"]

  example_movie_tor = table()
  example_movie_tor["magnets"] = ["mag2"]

  example_multifiles = table()
  example_multifiles["magnets"] = ["mag3", "mag4", "mag5", "..."]

  magnet_document.add(comment("################################################################################"))
  magnet_document.add(comment("#  This file explicitly outlines how torrents should be stored.                #"))
  magnet_document.add(comment("################################################################################"))
  magnet_document.add(comment("torrents stored in a folder or are a single file"))
  magnet_document.add("show1fldr", example_folder_tor)
  magnet_document.add("movie1", example_movie_tor)
  magnet_document.add(nl())
  magnet_document.add(comment("seperate file episodes"))
  magnet_document.add("show2fldr", example_multifiles)

  magnet_file.write(magnet_document)

# Download Torrents

In [13]:
# TODO: Handle torrent files
def download_torrent(progress: Progress, media_name: str, link: str):
  MAX_FILENAME_LEN = 35
  METADATA_TIMEOUT_MS = .8
  save_path=Path("/content/Torrents/") / media_name
  if not save_path.exists():
    save_path.mkdir(parents=True)

  task = progress.add_task(
        "download",
        total=100,
        status="[yellow]Obtaining Metadata",
        filename=media_name,
        speed="0.0 kB/s",
  )
  ses = lt.session()

  # setup torrent handle
  try:
    atp = lt.parse_magnet_uri(link)           # atp = add torrent parameters
  except Exception as e:
    progress.update(task, status="[red]Failed to parse magnet!")
    return None
  atp.save_path = str(save_path.as_posix())

  handle = ses.add_torrent(atp)
  status = handle.status()

  while not handle.status().has_metadata:
    time.sleep(METADATA_TIMEOUT_MS/1000)
    status = handle.status()

  filename =  status.name if len(status.name) < MAX_FILENAME_LEN \
              else f"{status.name[:MAX_FILENAME_LEN]}..."

  progress.update(task,
                  status="[yellow]Downloading",
                  filename=filename
  )

  # download torrent
  while not status.is_seeding:
    status = handle.status()

    progress.update(task,
                completed=status.progress * 100,
                speed=f"{status.download_rate/1000:.1f} kB/s",
    )
  progress.update(task,
                  status="[green]Complete",
                  completed=100,
  )

  return save_path

In [14]:
with Progress(
  TextColumn("{task.fields[status]}"),
  TextColumn("[bold]{task.fields[filename]}"),
  BarColumn(),
  TaskProgressColumn(),
  TimeRemainingColumn(),
  TextColumn("{task.fields[speed]}"),
) as progress:

  with ThreadPoolExecutor() as executor:
    media = magnet_file.read()
    futures = []
    for m in media:
      magnet_links = media[m]["magnets"]

      for link in magnet_links:
        futures.append(executor.submit(download_torrent, progress, m, link))

    for future in as_completed(futures):
      try:
          future.result()
      except Exception as e:
          print("Upload failed:", e)

Output()

# Upload to Dropbox
- plan to allow more cloud providers

In [26]:
from google.colab import drive
drive.mount('/content/drive')

from configparser import ConfigParser
from pathlib import Path

cfg = ConfigParser()

# observed that google has this file and copy that config file
gdrive_savepath = Path("/content/drive/MyDrive/Apps/torrent_dl/rclone/rclone.conf")
rclone_path = Path("/root/.config/rclone/rclone.conf")
if gdrive_savepath.exists():
  rclone_path.parent.mkdir(parents=True, exist_ok=True)
  !cp {gdrive_savepath.as_posix()} {rclone_path.as_posix()}

if not rclone_path.exists(): # create the config file if it doesn't exist
  rclone_path.parent.mkdir(parents=True, exist_ok=True)
  rclone_path.touch()

  cfg['dropbox'] =  \
  {
      "type": "dropbox",
      "token": ""
  }

  token = input("With rclone installed on a local system outside of google colab, run \"rclone authorize dropbox\" and paste the result here: ")
  cfg['dropbox']['token'] = token

  with rclone_path.open('w') as cfgwriter:
    cfg.write(cfgwriter)

  # save to google drive for future use
  if not gdrive_savepath.exists():
    gdrive_savepath.parent.mkdir(parents=True, exist_ok=True)
    gdrive_savepath.touch()

  with gdrive_savepath.open('w') as cfgwriter:
    cfg.write(cfgwriter)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
!rclone copy ./Torrents/ dropbox:/Anime/